In [ ]:
%matplotlib inline
import random

import numpy as np

from justice import xform
from justice import summarize
from justice import ogle_data
from justice import visualize

In [ ]:
cep_dataset = ogle_data.OgleDataset.read_for_name("cep")
df = cep_dataset.index_df

type_to_ids = {}
for key, subframe in df.sort_values('Subtype').groupby('Subtype'):
    type_to_ids.setdefault(key, [])
    type_to_ids[key].extend(sorted(frozenset(subframe.index.tolist())))

In [ ]:
# Choose 2 random light curves of the same type.

# The dataset generation may differ, so just hardcode the IDs for now.
random_type = 'F1'
random_ids = ['OGLE-LMC-CEP-0432', 'OGLE-LMC-CEP-0507']
assert frozenset(random_ids) <= frozenset(type_to_ids[random_type])

def load_for_id(random_id):
    lc = cep_dataset.lc_for_id(random_id)
    return lc.per_band_normalization(1000.0).transform(lc)

lca = load_for_id(random_ids[0])
lcb = load_for_id(random_ids[1])
visualize.plot_single_lc_color_bands(
    lca, "{}, type {}".format(random_ids[0], random_type)
)
visualize.plot_single_lc_color_bands(
   lcb, "{}, type {}".format(random_ids[1], random_type)
)
None  # Here and below: Avoid plot showing up twice by setting cell return value to None.

In [ ]:
# Currently seems to yield disjoint curves.
import importlib; importlib.reload(summarize)

# Desired solution
# aff = xform.Aff(tx=50)
aff = xform.Aff()

ivals = aff.as_array()
overlap_cost_fcn = summarize.OverlapCostComponent(
    np.array([1.0, 0.1, 0.05, 0.0]), 10.0)
aff = summarize.opt_gp(
    lca,
    lcb,
    ivals=ivals,
    vb=False,
    options={'maxiter': 15},
    overlap_cost_fcn=overlap_cost_fcn,
)
visualize.plot_lcs([lca, xform.transform(lcb, aff)])
aff